<a href="https://colab.research.google.com/github/DavidGlezGmz/Computer-Vision/blob/main/ImageProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
I = plt.imread('/content/drive/MyDrive/Colab Notebooks/Image Processing/bmwi4_50.jpg')

In [ ]:
plt.figure(figsize=(15,10))
plt.imshow(I)

In [ ]:
I.shape

In [ ]:
plt.imshow(I[:,:,0],cmap='Blues')

In [ ]:
plt.imshow(I[:,:,2],cmap='Greens')

In [ ]:
plt.imshow(I[:,:,1],cmap='Reds')


In [ ]:
BW=np.mean(I, axis=2)

In [ ]:
plt.imshow(BW, cmap='gray')

In [ ]:
BW.shape

In [ ]:
I_contrast = I**1.8/255

In [ ]:
plt.imshow(I_contrast)

In [ ]:
from PIL import Image

In [ ]:
Ip = Image.open('/content/drive/MyDrive/Colab Notebooks/Image Processing/bmwi4_50.jpg')

In [ ]:
Irez = Ip.resize((220,190),)

In [ ]:
Irez

In [ ]:
plt.plot(Ip.histogram())

In [ ]:
def discrete2DConvolution(Image, filter, padding):
  paddedImage = np.pad(Image, padding, "constant")
  flter = np.rot90(filter,2)
  conv = np.empty(Image.shape)

  for i in range(paddedImage.shape[0]- flter.shape[0]):
    for j in range(paddedImage.shape[1]-flter.shape[1]):
      conv[i,j] = np.sum((paddedImage[i:i+flter.shape[0], j:j+flter.shape[1]])*flter)
  
  return conv

In [ ]:
flterXX=np.array([[0,1,0],[0,1,0],[0,1,0]])
flterNeg=np.array([[-10,1,-1],[-1,1,-1],[-1,1,-1]])
flterREdge=np.array([[-10,0,10],[-10,0,10],[-10,0,10]])

In [ ]:
plt.imshow(flterREdge)

In [ ]:
flterLEdge = np.array([[10,0,-10],[10,0,-10],[10,0,-10]])
plt.imshow(flterLEdge)

In [ ]:
I_conv = discrete2DConvolution(BW, flterREdge, 1)

In [ ]:
plt.imshow(I_conv, cmap='gray')

In [ ]:
def MaxPool(Image, kSize, stride):
  n=0
  k=0
  vert = int(Image.shape[1]/stride)
  hoz = int (Image.shape[0]/stride)
  MaxPool = np.empty((hoz, vert))

  for i in range(0, Image.shape[0]-stride, stride):
    n=0
    for j in range(0, Image.shape[1]-stride, stride):
      MaxPool[k,n] = np.max(Image[i:i+kSize, j:j+kSize])
      n+=1
    k+=1
  
  return MaxPool

In [ ]:
ImPool = MaxPool(BW,2,2)
plt.figure(figsize=(15,10))
plt.imshow(ImPool, cmap='gray')

In [ ]:
ImPool = MaxPool(ImPool,2,1)
plt.figure(figsize=(15,10))
plt.imshow(ImPool, cmap='gray')

In [ ]:
ImPool_ledge = MaxPool(I_conv,4,4)
plt.figure(figsize=(15,10))
plt.imshow(ImPool_ledge, cmap='gray')

## Basic Cat and Dog Classifier in TF and Keras

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

In [ ]:
import os
import zipfile
local_zip = '/content/drive/MyDrive/Colab Notebooks/Enhance IT/Image Processing/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/drive/MyDrive/Colab Notebooks/Enhance IT/Image Processing')
zip_ref.close()

In [ ]:
base_dir='/content/drive/MyDrive/Colab Notebooks/Enhance IT/Image Processing/cats_and_dogs_filtered'
train_dir=os.path.join(base_dir,'train')
validation_dir=os.path.join(base_dir, 'validation')

In [ ]:
train_cats_dir=os.path.join(train_dir, 'cats')
train_dogs_dir=os.path.join(train_dir, 'dogs')
validation_cats_dir=os.path.join(validation_dir, 'cats')
validation_dogs_dir=os.path.join(validation_dir, 'dogs')

In [ ]:
import matplotlib.image as mpimg

In [ ]:
train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

nrows = 4
ncols = 4
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index = 8
next_cat = [os.path.join(train_cats_dir, fname) for fname in train_cat_fnames[pic_index-8:pic_index]]
next_dog = [os.path.join(train_dogs_dir, fname) for fname in train_dog_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_cat+next_dog):
  sp=plt.subplot(nrows, ncols, i+1)
  sp.axis('Off')
  img=mpimg.imread(img_path)
  plt.imshow(img)

## Build CNN in Keras

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
img_input = layers.Input(shape=(150,150,3))
L = layers.Conv2D(16,3, activation='relu')(img_input)
L = layers.MaxPooling2D(2)(L)

L = layers.Conv2D(32,3,activation='relu')(L)
L = layers.MaxPooling2D(2)(L)

L = layers.Conv2D(64,3,activation='relu')(L)
L = layers.MaxPooling2D(2)(L)

In [ ]:
# FULL CONECTION

F = layers.Flatten()(L)
F = layers.Dense(512, activation='relu')(F)
output = layers.Dense(1, activation='sigmoid')(F)

In [ ]:
# BUILD MODEL

model = Model(img_input, output)
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=0.001), metrics=['acc'])

In [ ]:
# Data Preparation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Rescaling

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='binary')

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50, verbose=2)

## Image Classification with VGG16 and Data Augmentation

In [ ]:
BATCH_Size = 20
IMG_SHAPE = 150 # Our training data consists of images with width of 150 pixels and height of 150 pixels

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
  fig, axes = plt.subplots(1,5,figsize=(20,20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
  plt.tight_layout()
  plt.show()


In [ ]:
image_gen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
train_data_gen = image_gen_train.flow_from_directory(
    batch_size=BATCH_Size,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_SHAPE,IMG_SHAPE),
    class_mode='binary'
)

In [ ]:
augmented_Im=[train_data_gen[0][0][0]for i in range(10)]
plotImages(augmented_Im)

In [ ]:
image_gen_val= ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(
    batch_size=BATCH_Size,
    directory = validation_dir,
    target_size=(IMG_SHAPE,IMG_SHAPE),
    class_mode = 'binary'
)

In [ ]:
INPUT_SHAPE=(150,150,3)

Loading VGG16

In [ ]:
from keras.applications import vgg16
import tensorflow as tf
import keras

In [ ]:
vgg = vgg16.VGG16(include_top = False, weights='imagenet', input_shape = INPUT_SHAPE)
output = vgg.layers[-1].output
output = tf.keras.layers.Flatten()(output)
vgg_model = Model(vgg.input, output)

In [ ]:
vgg_model.trainable = False
for layer in vgg_model.layers:
  layer.trainable = False

In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Flatten,Dropout,Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
model.summary()

In [ ]:
vgg_model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.0005),metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_data_gen, steps_per_epoch=100, epochs=10, validation_data = val_data_gen, validation_steps=50, callbacks=[callback],verbose=2)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, image

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(20, 12))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
def predict(file):
  img = image.load_img(file, target_size = (IMG_SHAPE, IMG_SHAPE))
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  print (img)


  #img = preprocess_input(img)
  probs = model.predict(img)
  print(probs)
  
  if probs[0][0]==1:
    result = 'dog'
  else: result = 'cat'

  pic = plt.imread(file)
  plt.imshow(pic)
  plt.show()
  print("This is a picture of a "+result+"!")
  return result

In [ ]:
predict('/content/drive/MyDrive/Colab Notebooks/Enhance IT/Image Processing/Dalmata.jpg')

In [ ]:
predict('/content/drive/MyDrive/Colab Notebooks/Enhance IT/Image Processing/Angry Dog.jpg')

In [ ]:
predict('/content/drive/MyDrive/Colab Notebooks/Enhance IT/Image Processing/cat-its-mouth-open.jpg')

In [ ]:
predict('/content/drive/MyDrive/Colab Notebooks/Enhance IT/Image Processing/egypt_kitty_social.jpg')